## Setup and helper functions

In [1]:
import pandas as pd
import rdflib
from rdflib import URIRef
import os
import csv
import pprint
import hashlib
pp = pprint.PrettyPrinter(indent=4)

data_or = 'data-original' + os.sep + 'phenotypic' + os.sep
data_gen = 'data-generated' + os.sep + 'phenotypic' + os.sep

if not os.path.exists('data-generated'):
    os.mkdir('data-generated')
if not os.path.exists(data_gen):
    os.mkdir(data_gen)

In [2]:
def reshape_sheet(sheet):
    new_sheet = sheet.where(pd.notnull(sheet), None)
    new_sheet = new_sheet.drop([0,1,2])
    del(new_sheet['Field'])
    new_sheet.reset_index(inplace=True, drop=True)
    return(new_sheet)

def make_safe(s):
    safe_s = ''
    for c in s:
        safe_s += c if c.isalnum() else '_'
    return safe_s

In [3]:
def parseStudyData(filename, df, nodes):
    g = rdflib.Graph()  # Create RDF graph  
    # Define prefixes
    rdf = rdflib.Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#')
    rdfs = rdflib.Namespace('http://www.w3.org/2000/01/rdf-schema#')
    ppeo = rdflib.Namespace('http://purl.org/ppeo/PPEO.owl#')
    g.bind('rdf', rdf)
    g.bind('rdfs', rdfs)
    g.bind('ppeo', ppeo)
    
    _, study_ID = filename.split('_')
    study_ID, _ = study_ID.split('.')
    fileNode = nodes['datafiles'][filename]
    varColumns = list(df.columns)
    varColumns.remove('Observation unit ID')
    varColumns.remove('Date')
    counter_obs_ID = 0
    for index, row in df.iterrows():
        curDate = rdflib.Literal(row['Date'], datatype=rdflib.XSD.date)
        curOu = nodes['ou'][row['Observation unit ID']]
        for var in varColumns: 
            # ignore if there is no value
            if row[var]:
                if var in nodes['ov']:
                    varNode = nodes['ov'][var.strip()]
                else:
                    print('Variable not found: ' + var)
                    continue
                #curVar = 1
                counter_obs_ID += 1
                obsID_base_string = ('#observation/' + nodes['investigation_ID'] + '_' + study_ID + 
                                     '_' + make_safe(var))
                obsNode = URIRef(obsID_base_string + '_' + str(counter_obs_ID))  # rdflib.BNode() 
                g.add((
                    obsNode,
                    rdf.type,
                    ppeo.observation
                ))
                g.add((
                    fileNode,
                    ppeo.hasObservation,
                    obsNode
                ))
                g.add((
                    fileNode,
                    ppeo.hasDigitalLocation,
                    rdflib.Literal(filename)
                ))
                g.add((
                    obsNode,
                    ppeo.hasObservedSubject,
                    curOu
                ))
                g.add((
                    obsNode,
                    ppeo.hasVariable,
                    varNode
                ))
                g.add((
                    obsNode,
                    ppeo.hasDateTime,
                    curDate
                ))
                g.add((
                    obsNode,
                    ppeo.hasValue,
                    rdflib.Literal(row[var])
                ))
    return g

## MIAPPE metadata to ttl

In [4]:
# Read sheets from file
inv_sheet = pd.read_excel('data-original' + os.sep + 'MIAPPE_CxE_v1.1.xlsx', sheet_name='Investigation')
study_sheet = pd.read_excel('data-original' + os.sep + 'MIAPPE_CxE_v1.1.xlsx', sheet_name='Study')
person_sheet = pd.read_excel('data-original' + os.sep + 'MIAPPE_CxE_v1.1.xlsx', sheet_name='Person')
datafile_sheet = pd.read_excel('data-original' + os.sep + 'MIAPPE_CxE_v1.1.xlsx', sheet_name='Data file')
bm_sheet = pd.read_excel('data-original' + os.sep + 'MIAPPE_CxE_v1.1.xlsx', sheet_name='Biological Material')
ou_sheet = pd.read_excel('data-original' + os.sep + 'MIAPPE_CxE_v1.1.xlsx', sheet_name='Observation Unit')
ov_sheet = pd.read_excel('data-original' + os.sep + 'MIAPPE_CxE_v1.1.xlsx', sheet_name='Observed Variable')

# drop unnecessary columns/rows
study_sheet = reshape_sheet(study_sheet)
person_sheet = reshape_sheet(person_sheet)
datafile_sheet = reshape_sheet(datafile_sheet)
bm_sheet = reshape_sheet(bm_sheet)
ou_sheet = reshape_sheet(ou_sheet)
ov_sheet = reshape_sheet(ov_sheet)


In [5]:
# Create graph, parse investigation

nodes = {}

# Define prefixes
rdf = rdflib.Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#')
rdfs = rdflib.Namespace('http://www.w3.org/2000/01/rdf-schema#')
ppeo = rdflib.Namespace('http://purl.org/ppeo/PPEO.owl#')
base = rdflib.Namespace('http://localhost/wur/fair-example#')

g = rdflib.Graph()  # Create RDF graph

g.bind('rdf', rdf)
g.bind('rdfs', rdfs)
g.bind('ppeo', ppeo)

# Investigation
inv_ID = 'phInvID'
if inv_sheet[['Value'][0]][0]:
    inv_ID = make_safe(inv_sheet[['Value'][0]][0].strip())
else:
    print('INFO: no investigation ID')

invNode = URIRef('#investigation/' + make_safe(inv_ID))  # rdflib.BNode()
nodes = {'investigation': invNode, 'investigation_ID': make_safe(inv_ID)}
g.add((
    invNode,
    rdf.type,
    ppeo.investigation
))    
    
g.add((
    invNode,
    ppeo.hasIdentifier,
    rdflib.Literal(inv_ID)
))
    
if inv_sheet[['Value'][0]][1]:    
    g.add((
        invNode,
        ppeo.hasName,
        rdflib.Literal(inv_sheet[['Value'][0]][1])
    ))
else:
    print('ERROR: no investigation title')

if inv_sheet[['Value'][0]][2]:  
    g.add((
        invNode,
        ppeo.hasDescription,
        rdflib.Literal(inv_sheet[['Value'][0]][2])
    ))
else:
    print('INFO: no investigation description')
    
if inv_sheet[['Value'][0]][3]:  
    g.add((
        invNode,
        ppeo.hasSubmissionDate,
        rdflib.Literal(inv_sheet[['Value'][0]][3], datatype=rdflib.XSD.date)
    ))
else:
    print('INFO: no investigation submission date')   
    
if inv_sheet[['Value'][0]][4]:  
    g.add((
        invNode,
        ppeo.hasPublicReleaseDate,
        rdflib.Literal(inv_sheet[['Value'][0]][4], datatype=rdflib.XSD.date)
    ))
else:
    print('INFO: no investigation release date')
    
if inv_sheet[['Value'][0]][5]:  
    g.add((
        invNode,
        ppeo.hasLicense,
        rdflib.Literal(inv_sheet[['Value'][0]][5])
    ))
else:
    print('INFO: no investigation license')

if inv_sheet[['Value'][0]][6]:  
    g.add((
        invNode,
        ppeo.hasMIAPPEVersion,
        rdflib.Literal(inv_sheet[['Value'][0]][6], datatype=rdflib.XSD.float)
    ))
else:
    print('INFO: no investigation miappe version')
    
if inv_sheet[['Value'][0]][7]:  
    g.add((
        invNode,
        ppeo.hasAssociatedPublication,
        rdflib.Literal(inv_sheet[['Value'][0]][7])
    ))
else:
    print('INFO: no investigation publications')
    

In [6]:
# Study
nodes['studies'] = {}
nodes['obsLevelHierarchy'] = {}
nodes['spatDist'] = {}
for study in study_sheet['Study unique ID*']:
    stuIDstp = study.strip()
    nodes['studies'][stuIDstp] = URIRef('#study/' + inv_ID + '_' + make_safe(stuIDstp))  # rdflib.BNode()
    nodes['obsLevelHierarchy'][stuIDstp] = {}
    nodes['spatDist'][stuIDstp] = {'study': '#spatial_distribution/' + inv_ID + '_' + 
                                   make_safe(stuIDstp) + '_spatial_distribution'}
    
nodes['countries'] = {}  
nodes['locations'] = {}
locationColumns = zip(
    study_sheet['Geographic location (country)*'], 
    study_sheet['Experimental site name*'],
    study_sheet['Geographic location (latitude)'], 
    study_sheet['Geographic location (longitude)'], 
    study_sheet['Geographic location (altitude)']
)
for countryStr, locStr, latStr, lonStr, altStr in locationColumns:
    country = countryStr.strip()
    if country not in nodes['countries']:
        nodes['countries'][country] = URIRef('#country/' + make_safe(country))  # rdflib.BNode() 
        g.add((
            nodes['countries'][country],
            ppeo.hasName,
            rdflib.Literal(country)
        ))
        g.add((
            nodes['countries'][country],
            rdf.type,
            ppeo.country
        ))
    
    loc = locStr.strip()
    if loc not in nodes['locations']:
        loc_hash = int(hashlib.sha1(loc.encode('utf-8')).hexdigest(), 16) % (10 ** 8)
        nodes['locations'][loc] = URIRef('#location/' + str(loc_hash))  # rdflib.BNode() 
        g.add((
            nodes['locations'][loc],
            rdf.type,
            ppeo.location
        ))
        g.add((
            nodes['locations'][loc],
            ppeo.hasCountry,
            nodes['countries'][country]
        ))
        locParts = [x.strip() for x in loc.split(';')]

        g.add((
            nodes['locations'][loc],
            ppeo.hasName,
            rdflib.Literal(locParts[1])
        ))
        g.add((
            nodes['locations'][loc],
            ppeo.hasAddress,
            rdflib.Literal(locParts[0])
        ))

        lat, long, alt = float(latStr), float(lonStr), altStr.strip()
        g.add((
            nodes['locations'][loc],
            ppeo.hasLatitude,
            rdflib.Literal(lat, datatype=rdflib.XSD.float)
        ))
        g.add((
            nodes['locations'][loc],
            ppeo.hasLongitude,
            rdflib.Literal(long, datatype=rdflib.XSD.float)
        ))            
        g.add((
            nodes['locations'][loc],
            ppeo.hasAltitude,
            rdflib.Literal(alt, datatype=rdflib.XSD.string)
        ))       
        
nodes['institutes'] = {}
for instStr in study_sheet['Contact institution*']:
    inst = instStr.strip()
    inst_hash = int(hashlib.sha1(inst.encode('utf-8')).hexdigest(), 16) % (10 ** 8)
    if inst not in nodes['institutes']:
        nodes['institutes'][inst] = URIRef('#institute/' + str(inst_hash)) #rdflib.BNode() 
        instParts = [x.strip() for x in inst.split(';')]
        g.add((
            nodes['institutes'][inst],
            ppeo.hasName,
            rdflib.Literal(instParts[0])
        ))
        g.add((
            nodes['institutes'][inst],
            rdf.type,
            ppeo.institution
        ))
        inst_loc_hash = int(hashlib.sha1((inst + ' ' + 'location').encode('utf-8')).hexdigest(), 16) % (10 ** 8)                            
        instLocNode = URIRef('#location/' + str(inst_loc_hash))  # rdflib.BNode() 
        g.add((
            nodes['institutes'][inst],
            ppeo.hasLocation,
            instLocNode
        ))
        g.add((
            instLocNode,
            rdf.type,
            ppeo.location
        )) 
        g.add((
            instLocNode,
            ppeo.hasName,
            rdflib.Literal(instParts[0])
        ))
        g.add((
            instLocNode,
            ppeo.hasAddress,
            rdflib.Literal(instParts[1])
        ))
        if instParts[2] in nodes['countries']:
            g.add((
                instLocNode,
                ppeo.hasCountry,
                nodes['countries'][instParts[2]]
            ))  
        else:
            print('Country ' + instParts[2] + ' for ' + instStr + ' not found: not implemented.')

            
for index, row in study_sheet.iterrows():
    #print(row['Study unique ID*'], row['Study title*'])
    
    stuIDstp = row['Study unique ID*'].strip()
    curStudyNode = nodes['studies'][stuIDstp]
    curStudyID = make_safe(stuIDstp)
    
    g.add((
        nodes['investigation'],
        ppeo.hasPart,
        curStudyNode
    ))
    g.add((
        curStudyNode,
        rdf.type,
        ppeo.study
    ))
    g.add((
        curStudyNode,
        ppeo.hasIdentifier,
        rdflib.Literal(row['Study unique ID*'])
    ))
    g.add((
        curStudyNode,
        ppeo.hasName,
        rdflib.Literal(row['Study title*'])
    ))
    
    curElem = row['Study description']
    if curElem and curElem.strip():
        g.add((
            curStudyNode,
            ppeo.hasDescription,
            rdflib.Literal(curElem)
        ))
    
    g.add((
        curStudyNode,
        ppeo.hasStartDateTime,
        rdflib.Literal(row['Start date of study*'], datatype=rdflib.XSD.date)
    ))
    g.add((
        curStudyNode,
        ppeo.hasEndDateTime,
        rdflib.Literal(row['End date of study'], datatype=rdflib.XSD.date)
    ))
    g.add((
        curStudyNode,
        ppeo.hasContactInstitution,
        rdflib.Literal(row['Contact institution*'])
    ))

    g.add((
        curStudyNode,
        ppeo.hasLocation,
        nodes['locations'][row['Experimental site name*'].strip()]
    ))

    g.add((
        curStudyNode,
        ppeo.hasDescriptionOfExperimentalDesign,
        rdflib.Literal(row['Description of statistical design*'])
    ))
    
    hierarchyText = rdflib.Literal(row['Observation unit level hierarchy']).strip()
    if hierarchyText:
        olhNode = URIRef('#observation_level_hierarchy/' + inv_ID + '_' + curStudyID + '_olh')  # rdflib.BNode() 
        nodes['obsLevelHierarchy'][stuIDstp]['hierarchy'] = olhNode
        g.add((
            curStudyNode,
            ppeo.hasObservationUnitLevelHierarchy,
            olhNode
        ))
        g.add((
            olhNode,
            rdf.type,
            ppeo.observation_level_hierarchy
        ))
        
        levels = [x.strip() for x in hierarchyText.split('>')]
        levelNode =  URIRef('#observation_level/' + inv_ID + '_' + curStudyID + '_' + levels[0])  # rdflib.BNode()
        nodes['obsLevelHierarchy'][stuIDstp][levels[0]] = levelNode
        g.add((
            levelNode,
            ppeo.partOf,
            olhNode
        ))
        g.add((
            levelNode,
            rdf.type,
            ppeo.observation_level
        ))
        g.add((
            levelNode,
            ppeo.hasType,
            rdflib.Literal(levels[0])
        ))
        previousLevel = levelNode
        for L in levels[1:]:
            levelNode = URIRef('#observation_level/' + inv_ID + '_' + curStudyID + '_' + L)  # rdflib.BNode() 
            nodes['obsLevelHierarchy'][stuIDstp][L] = levelNode
            g.add((
                levelNode,
                ppeo.partOf,
                olhNode
            ))
            g.add((
                levelNode,
                rdf.type,
                ppeo.observation_level
            ))
            g.add((
                levelNode,
                ppeo.hasType,
                rdflib.Literal(L)
            ))
            g.add((
                levelNode,
                ppeo.hasSuperlevel,
                previousLevel
            ))
    
    g.add((
        curStudyNode,
        ppeo.hasObservationUnitDescription,
        rdflib.Literal(row['Observation unit description*'])
    ))
    g.add((
        curStudyNode,
        ppeo.hasDescriptionOfGrowthFacility,
        rdflib.Literal(row['Description of growth facility*'])
    ))
    g.add((
        curStudyNode,
        ppeo.hasTypeOfGrowthFacility,
        rdflib.Literal(row['Type of growth facility'])
    ))
    g.add((
        curStudyNode,
        ppeo.hasCulturalPractices,
        rdflib.Literal(row['Cultural practices'])
    ))
    

# print(g.serialize(format='ttl').decode('utf8'))    

In [7]:
# Person
nodes['persons'] = {}
for person in person_sheet['Person name*']:
    if not person: continue
    nodes['persons'][person] = URIRef('#person/' + make_safe(person)) #rdflib.BNode()

for index, row in person_sheet.iterrows():
    if not (row['Person role*'] and row['Person name*']): break
    curPersonNode = nodes['persons'][row['Person name*']]
    person_roles = [x.strip() for x in row['Person role*'].split(';')]
    for pr in person_roles:
        #if not pr: break
        curRoleNode = URIRef('#person_role/' + inv_ID + '_' + make_safe(curStudyID) + 
                             '_' + make_safe(pr)) # rdflib.BNode()
        g.add((
            nodes['investigation'],
            ppeo.hasPersonWithRole,
            curRoleNode
        ))
        
        g.add((
            curRoleNode,
            ppeo.hasName,
            rdflib.Literal(pr)
        ))

        g.add((
            curRoleNode,
            ppeo.isRoleOf,
            curPersonNode,
        ))

    g.add((
        curPersonNode,
        rdf.type,
        ppeo.person
    ))
    g.add((
        curPersonNode,
        ppeo.hasName,
        rdflib.Literal(row['Person name*'])
    ))
    g.add((
        curPersonNode,
        ppeo.hasEmail,
        rdflib.Literal(row['Person email'])
    ))
    
    curElem = row['Person ID']
    if curElem and curElem.strip():
        g.add((
            curPersonNode,
            ppeo.hasIdentifier,
            rdflib.Literal(curElem)
        ))

    g.add((
        curPersonNode,
        ppeo.hasAffiliation,
        rdflib.Literal(row['Person affiliation*'])
    ))

In [8]:
# Data file

nodes['datafiles'] = {}
for index, row in datafile_sheet.iterrows():
    filename = row['Data file link*'].strip()
    fileNode = URIRef('#data_file/' + inv_ID + '_' + make_safe(filename))  # rdflib.BNode() 
    nodes['datafiles'][filename] = fileNode
    studyNode = nodes['studies'][row['Study unique ID*'].strip()]

    g.add((
        studyNode,
        ppeo.hasDataFile,
        fileNode
    ))
    g.add((
        fileNode,
        rdf.type,
        ppeo.data_file
    ))
    g.add((
        fileNode,
        ppeo.hasDigitalLocation,
        rdflib.Literal(filename)
    ))
    g.add((
        fileNode,
        ppeo.hasDescription,
        rdflib.Literal(row['Data file description*'].strip())
    ))
    g.add((
        fileNode,
        ppeo.hasVersion,
        rdflib.Literal(row['Data file version*'])
    ))

In [9]:
# Biological material

nodes['materials'] = {}
#for study in study_sheet['Study unique ID*']:
#    nodes['studies'][study] = rdflib.BNode()
# pp.pprint(nodes)

nodes['bm'] = {}
for bm in bm_sheet['Biological material ID*']:
    nodes['bm'][bm] = URIRef('#biological_material/' + inv_ID + '_' + make_safe(bm.strip()))  # rdflib.BNode()

nodes['ms'] = {}
for index, row in bm_sheet.iterrows():

    curBmNode = nodes['bm'][row['Biological material ID*']]
    studies_for_bm = [x.strip() for x in row['Study unique ID*'].split(';')]

    for s in studies_for_bm:
        g.add((
            nodes['studies'][s],
            ppeo.hasBiologicalMaterial,
            curBmNode
        ))
    g.add((
        curBmNode,
        rdf.type,
        ppeo.biological_material
    ))
    g.add((
        curBmNode,
        ppeo.hasIdentifier,
        rdflib.Literal(row['Biological material ID*'])
    ))
    g.add((
        curBmNode,
        ppeo.hasTaxonIdentifier,
        rdflib.Literal(row['Organism*'])
    ))
    g.add((
        curBmNode,
        ppeo.hasGenus,
        rdflib.Literal(row['Genus'])
    ))
    g.add((
        curBmNode,
        ppeo.hasSpecies,
        rdflib.Literal(row['Species'])
    ))
    
    curElem = row['Material source ID (Holding institute/stock centre, accession)']
    if curElem and curElem.strip():
        if curElem not in nodes['ms']:
            nodes['ms'][curElem] = URIRef('material_source/' + inv_ID + '_' + 
                                          make_safe(curElem.strip()))  # rdflib.BNode()
            curElemNode = nodes['ms'][curElem]
            g.add((
               curElemNode,
                rdf.type,
                ppeo.material_source
            ))
            g.add((
                curElemNode,
                ppeo.hasIdentifier,
                rdflib.Literal(curElem)
            ))
            g.add((
                curElemNode,
                ppeo.hasDOI,
                rdflib.Literal(row['Material source DOI'])
            ))
            g.add((
                curElemNode,
                ppeo.hasDescription,
                rdflib.Literal(row['Material source description'])
            ))
        curElemNode = nodes['ms'][curElem]    
        g.add((
            curBmNode,
            ppeo.hasMaterialSource,
            curElemNode
        ))

In [10]:
# Observation unit

nodes['ou'] = {}
for ou in ou_sheet['Observation unit ID*']:
    nodes['ou'][ou] = URIRef('#observation_unit/' + inv_ID + '_' + make_safe(ou.strip()))  # rdflib.BNode()

sd_counter = 0
for index, row in ou_sheet.iterrows():

    curOuNode = nodes['ou'][row['Observation unit ID*']]
    stuIDstp = row['Study unique ID*'].strip()
    curStudyID = make_safe(stuIDstp)
    g.add((
        curOuNode,
        rdf.type,
        ppeo.observation_unit
    ))
    g.add((
        curOuNode,
        ppeo.hasIdentifier,
        rdflib.Literal(row['Observation unit ID*'])       
    ))
    g.add((
        curOuNode,
        ppeo.partOf,
        nodes['studies'][stuIDstp]
    ))
    g.add((
        nodes['studies'][row['Study unique ID*'].strip()],
        ppeo.hasPart,
        curOuNode        
    ))
    
    if '#' in row['Biological Material ID*']:
        print(row['Observation unit ID*'])
    g.add((
        curOuNode,
        ppeo.hasBiologicalMaterial,
        nodes['bm'][row['Biological Material ID*'].strip()]
    ))
    
    ouType = row['Observation unit type*'].strip()
    if ouType in nodes['obsLevelHierarchy'][stuIDstp]:
        typeNode = nodes['obsLevelHierarchy'][stuIDstp][ouType]
        g.add((
            curOuNode,
            ppeo.hasType,
            typeNode
        ))
    else: 
        print('thisError: observation unit type ' + ouType + ' not found in study ' + stuIDstp)

    
    if row['Spatial distribution']:
        #print('found SD: ' + row['Spatial distribution'] )
        spatDistParts = row['Spatial distribution'].split(';')
        for sdPart in spatDistParts:
            parts = [x.strip() for x in sdPart.split(':')]
            #print(parts)
            if len(parts) < 2: 
                continue
                print('skipped spatial distribution')
            sdType, sdVal = parts[0], parts[1]
            #print(sdType, sdVal)
            
            if sdType in nodes['obsLevelHierarchy'][stuIDstp]:
                typeNode = nodes['obsLevelHierarchy'][stuIDstp][sdType]
            elif sdType in nodes['spatDist'][stuIDstp]:
                typeNode = nodes['spatDist'][stuIDstp][sdType]
            else: 
                typeNode = URIRef('#spatial_distribition_type/' + inv_ID + '_' + 
                                  curStudyID + make_safe(sdType))  # rdflib.BNode()
                nodes['spatDist'][stuIDstp][sdType] = typeNode
                g.add((
                    typeNode,
                    rdf.type,
                    ppeo.spatial_distribution_type
                ))
                g.add((
                    typeNode,
                    ppeo.hasType,
                    rdflib.Literal(sdType)
                ))
                
            curSdNode = URIRef('#spatial_distribution/' + inv_ID + '_' + 
                                curStudyID + '_' + make_safe(row['Observation unit ID*']) + '_' + str(sd_counter))  # rdflib.BNode()
            sd_counter += 1
            
            g.add((
                curOuNode,
                ppeo.hasSpatialDistribution,
                curSdNode
            ))
            g.add((
                curSdNode,
                ppeo.hasSpatialDistributionType,
                typeNode
            ))
            g.add((
                curSdNode,
                ppeo.hasValue,
                rdflib.Literal(sdVal)
            ))


In [11]:
# Observed variables

inv_ID = nodes['investigation_ID']
nodes['ov'] = {}
nodes['trait'] = {}
nodes['method'] = {}
nodes['scale'] = {}
for ov in ov_sheet['Variable ID*']:
    nodes['ov'][ov] = URIRef('#variable/' + inv_ID + '_' + make_safe(ov.strip()))  # rdflib.BNode()

for index, row in ov_sheet.iterrows():

    curOvNode = nodes['ov'][row['Variable ID*']]
    studies_for_ov = [x.strip() for x in row['Study unique ID*'].split(';')]

    for s in studies_for_ov:
        g.add((
            nodes['studies'][s],
            ppeo.hasPart,
            curOvNode
        ))
    g.add((
        curOvNode,
        rdf.type,
        ppeo.observed_variable
    ))
    g.add((
        curOvNode,
        ppeo.hasIdentifier,
        rdflib.Literal(row['Variable ID*'])
    ))
    
    t = row['Trait*'].strip()
    tAc = ''
    if row['Trait accession number']:
        tAc = row['Trait accession number'].strip()
    if t in nodes['trait']:
        tNode = nodes['trait'][t]
    else:
        tHash = int(hashlib.sha1(t.encode('utf-8')).hexdigest(), 16) % (10 ** 8)
        tNode = URIRef('#trait/' + inv_ID + '_' + str(tHash))  # rdflib.BNode()
        g.add((
            tNode,
            ppeo.hasName,
            rdflib.Literal(t)
        ))
        if tAc:
            g.add((
                tNode,
                ppeo.hasTraitAccessionNumber,
                rdflib.Literal(tAc)
            ))
        
    g.add((
        curOvNode,
        ppeo.hasTrait,
        tNode
    ))

    m = row['Method*'].strip()
    mAc = ''
    if row['Method accession number']:
        mAc = row['Method accession number'].strip()
    if m in nodes['method']:
        mNode = nodes['method'][m]
    else:
        mHash = int(hashlib.sha1(m.encode('utf-8')).hexdigest(), 16) % (10 ** 8)
        mNode = URIRef('#method/' + inv_ID + '_' + str(mHash))  # rdflib.BNode()
        nodes['method'][m] = mNode
        g.add((
            mNode,
            ppeo.hasName,
            rdflib.Literal(m)
        ))
        if mAc:
            g.add((
                mNode,
                ppeo.hasMethodAccessionNumber,
                rdflib.Literal(mAc)
            ))
        
    g.add((
        curOvNode,
        ppeo.hasMethod,
        mNode
    ))    
    
    
    s = row['Scale*'].strip()
    sAc = ''
    if row['Scale accession number']:
        sAc = row['Scale accession number'].strip()
    if s in nodes['scale']:
        sNode = nodes['scale'][s]
    else:
        sHash = int(hashlib.sha1(s.encode('utf-8')).hexdigest(), 16) % (10 ** 8)
        sNode = URIRef('#scale/' + inv_ID + '_' + str(sHash))  # rdflib.BNode()
        nodes['scale'][s] = sNode
        g.add((
            sNode,
            ppeo.hasName,
            rdflib.Literal(s)
        ))
        if sAc:
            g.add((
                sNode,
                ppeo.hasMethodAccessionNumber,
                rdflib.Literal(sAc)
            ))
        
    g.add((
        curOvNode,
        ppeo.hasScale,
        sNode
    )) 


In [12]:
# print(g.serialize(format='ttl').decode('utf8')) 
g.serialize(destination='data-generated' + os.sep + 'miappe_metadata.ttl', 
            base='http://localhost:3030/wur/fair-example', 
            format='turtle')

## Transform actual data

### Netherlands 1999 data to ttl

In [13]:
df_1999NL =  pd.read_csv(data_or + 'data_1999NL.csv', sep='\t')
df_1999NL = df_1999NL.where(pd.notnull(df_1999NL), None)
df_1999NL['Date'] = pd.to_datetime(df_1999NL['Date']).dt.date

In [14]:
# look at data info
df_1999NL;
df_1999NL.dtypes;
# df_1999NL.info();
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(df_1999NL)
'ok'
# examining missing values
print('Are there any missing values? ', df_1999NL.isnull().values.any())
print('Are there any missing values in the Date column? ', df_1999NL['Date'].isnull().values.any())
# print('How many missing values are there per column?\n', df_1999NL.isnull().sum())
print('How many missing values are there overall? ', df_1999NL.isnull().sum().sum())
total_cells = len(df_1999NL.columns) * len(df_1999NL)
print('Missing value percentage: ' + str( 
      round(df_1999NL.isnull().sum().sum() * 100 / total_cells,2)) + '%')

Are there any missing values?  True
Are there any missing values in the Date column?  False
How many missing values are there overall?  1015
Missing value percentage: 35.1%


In [15]:
# g.remove((None, None, None))
g = parseStudyData('data_1999NL.csv', df_1999NL, nodes)
#print(g_1999NL.serialize(format='ttl').decode('utf8'))
g.serialize(destination=data_gen + 'data_1999NL.ttl', format='ttl')

### Venezuela 2003 data to ttl

In [16]:
df_2003VE =  pd.read_csv(data_or + 'data_2003VE.csv', sep='\t')
df_2003VE = df_2003VE.where(pd.notnull(df_2003VE), None)
df_2003VE['Date']= pd.to_datetime(df_2003VE['Date']).dt.date
df_2003VE

,Observation unit ID,Date,TubN_average_per_genotype,TubW_average_per_genotype
0,WUR:CE-VEN03-CE011,2003-11-30,18.5,142.5
1,WUR:CE-VEN03-CE032,2003-11-30,8.0,237.5
2,WUR:CE-VEN03-CE047,2003-11-30,21.0,230.0
3,WUR:CE-VEN03-CE067,2003-11-30,11.5,62.5
4,WUR:CE-VEN03-CE070,2003-11-30,7.0,125.0
5,WUR:CE-VEN03-CE100,2003-11-30,8.0,127.5
6,WUR:CE-VEN03-CE102,2003-11-30,16.0,176.0
7,WUR:CE-VEN03-CE110,2003-11-30,9.5,56.0
8,WUR:CE-VEN03-CE155,2003-11-30,27.0,100.0
9,WUR:CE-VEN03-CE159,2003-11-30,22.0,272.5


In [17]:
# look at data info
df_2003VE;
df_2003VE.dtypes;
# df_2003VE.info();
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(df_1999NL)

# examining missing values
print('Are there any missing values? ', df_2003VE.isnull().values.any())
print('Are there any missing values in the Date column? ',df_2003VE['Date'].isnull().values.any())
# print('How many missing values are there per column?\n', df_2003VE.isnull().sum())
print('How many missing values are there overall? ', df_2003VE.isnull().sum().sum())
total_cells = len(df_2003VE.columns) * len(df_2003VE)
print('Missing value percentage: ' + str( 
      round(df_2003VE.isnull().sum().sum() * 100 / total_cells,2)) + '%')

# g.remove((None, None, None))
g = parseStudyData('data_2003VE.csv', df_2003VE, nodes)
#print(g_1999NL.serialize(format='ttl').decode('utf8'))
g.serialize(destination=data_gen + 'data_2003VE.ttl', format='ttl')

Are there any missing values?  False
Are there any missing values in the Date column?  False
How many missing values are there overall?  0
Missing value percentage: 0.0%


### Ethiopia 2010 data to ttl

In [18]:
df_2010ET =  pd.read_csv(data_or + 'data_2010ET.csv', sep='\t')
df_2010ET = df_2010ET.where(pd.notnull(df_2010ET), None)
df_2010ET['Date']= pd.to_datetime(df_2010ET['Date']).dt.date
df_2010ET

,Observation unit ID,Date,TubN_LT20,TubW_LT20,TubN_GE20,TubW_GE20,TubN_total_per_plant,TubW_total_per_plant
0,WUR:CE-ETH10-1-1-1,2010-12-01,10,140,10,410,20,550
1,WUR:CE-ETH10-1-1-2,2010-12-01,2,46,3,145,5,191
2,WUR:CE-ETH10-1-1-3,2010-12-01,5,75,9,460,14,535
3,WUR:CE-ETH10-1-1-4,2010-12-01,3,65,4,185,7,250
4,WUR:CE-ETH10-1-2-1,2010-12-01,5,86,6,255,11,341
5,WUR:CE-ETH10-1-2-2,2010-12-01,7,80,16,705,23,785
6,WUR:CE-ETH10-1-2-3,2010-12-01,10,143,9,340,19,483
7,WUR:CE-ETH10-1-2-4,2010-12-01,4,85,4,280,8,365
8,WUR:CE-ETH10-1-3-1,2010-12-01,16,104,9,305,25,409
9,WUR:CE-ETH10-1-3-2,2010-12-01,11,110,3,95,14,205


In [19]:
# look at data info
df_2010ET;
df_2010ET.dtypes;
# df_2003VE.info();
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(df_1999NL)

# examining missing values
print('Are there any missing values? ', df_2010ET.isnull().values.any())
print('Are there any missing values in the Date column? ',df_2010ET['Date'].isnull().values.any())
print('How many missing values are there per column?\n', df_2010ET.isnull().sum())
missing = df_2010ET.isnull().sum().sum()
print('How many missing values are there overall? ', missing)
total_cells = len(df_2010ET.columns) * len(df_2010ET)
print('Total cells: ' + str(total_cells))
print('Existing values: ' + str(total_cells - missing))
print('Missing value percentage: ' + str( 
      round(missing * 100 / total_cells,2)) + '%')

# g.remove((None, None, None))
g = parseStudyData('data_2010ET.csv', df_2010ET, nodes)
#print(g_1999NL.serialize(format='ttl').decode('utf8'))
g.serialize(destination=data_gen + 'data_2010ET.ttl', format='ttl')

Are there any missing values?  True
Are there any missing values in the Date column?  False
How many missing values are there per column?
 Observation unit ID       0
Date                      0
TubN_LT20               298
TubW_LT20               300
TubN_GE20               110
TubW_GE20               112
TubN_total_per_plant      0
TubW_total_per_plant      0
dtype: int64
How many missing values are there overall?  820
Total cells: 18432
Existing values: 17612
Missing value percentage: 4.45%


### Finland 2004 data to ttl

In [20]:
df_2004Fin =  pd.read_csv(data_or + 'data_2004Fin.csv', sep='\t')
df_2004Fin = df_2004Fin.where(pd.notnull(df_2004Fin), None)
df_2004Fin['Date']= pd.to_datetime(df_2004Fin['Date']).dt.date
df_2004Fin

,Observation unit ID,Date,DoE,NumMainStems,PlantHeight,TubSizeN_12-20,TubSizeN_21-30,TubSizeN_31-40,TubSizeN_41-50,TubSizeN_51-60,TubSizeN_61-70,TubSizeW_12-20,TubSizeW_21-30,TubSizeW_31-40,TubSizeW_41-50,TubSizeW_51-60,TubSizeW_61-70,TubN_total_per_plant,TubW_total_per_plant
0,WUR:CE-FIN04-1-1-A,2004-06-30,16,3,7,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,WUR:CE-FIN04-1-1-B,2004-06-30,19,4,9,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,WUR:CE-FIN04-1-1-C,2004-06-30,19,2,6,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,WUR:CE-FIN04-1-2-A,2004-06-30,18,3,9.5,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,WUR:CE-FIN04-1-2-B,2004-06-30,18,3,11,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,WUR:CE-FIN04-1-2-C,2004-06-30,17,5,12,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,WUR:CE-FIN04-1-3-A,2004-06-30,14,4,8.5,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,WUR:CE-FIN04-1-3-B,2004-06-30,15,4,6.5,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,WUR:CE-FIN04-1-3-C,2004-06-30,15,4,6,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,WUR:CE-FIN04-1-4-A,2004-06-30,27,1,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [21]:
# look at data info
df_2004Fin;
df_2004Fin.dtypes;
df_2004Fin.info();
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(df_2004FinNL)

# examining missing values
print('Are there any missing values? ', df_2004Fin.isnull().values.any())
print('Are there any missing values in the Date column? ',df_2004Fin['Date'].isnull().values.any())
print('How many missing values are there per column?\n', df_2004Fin.isnull().sum())
missing = df_2004Fin.isnull().sum().sum()
print('How many missing values are there overall? ', missing)
total_cells = len(df_2004Fin.columns) * len(df_2004Fin)
print('Total cells: ' + str(total_cells))
print('Existing values: ' + str(total_cells - missing))
print('Missing value percentage: ' + str( 
      round(missing * 100 / total_cells,2)) + '%')

# g.remove((None, None, None))
g = parseStudyData('data_2004Fin.csv', df_2004Fin, nodes)
#print(g_2004Fin.serialize(format='ttl').decode('utf8'))
g.serialize(destination=data_gen + 'data_2004Fin.ttl', format='ttl')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4629 entries, 0 to 4628
Data columns (total 19 columns):
Observation unit ID     4629 non-null object
Date                    4629 non-null object
DoE                     4541 non-null object
NumMainStems            4542 non-null object
PlantHeight             3902 non-null object
TubSizeN_12-20          565 non-null object
TubSizeN_21-30          627 non-null object
TubSizeN_31-40          623 non-null object
TubSizeN_41-50          513 non-null object
TubSizeN_51-60          199 non-null object
TubSizeN_61-70          25 non-null object
TubSizeW_12-20          651 non-null object
TubSizeW_21-30          651 non-null object
TubSizeW_31-40          651 non-null object
TubSizeW_41-50          651 non-null object
TubSizeW_51-60          651 non-null object
TubSizeW_61-70          651 non-null object
TubN_total_per_plant    651 non-null object
TubW_total_per_plant    651 non-null object
dtypes: object(19)
memory usage: 343.6+ KB
Are there 

### Finland 2005 data to ttl

In [22]:
df_2005Fin =  pd.read_csv(data_or + 'data_2005Fin.csv', sep='\t')
df_2005Fin = df_2005Fin.where(pd.notnull(df_2005Fin), None)
df_2005Fin['Date']= pd.to_datetime(df_2005Fin['Date']).dt.date
df_2005Fin

,Observation unit ID,Date,NumMainStems,TubN_total_per_plant,TubW_total_per_plant,DoE,PlantHeight
0,WUR:CE-FIN05-I-1-A,2005-06-28,None,None,None,32,6
1,WUR:CE-FIN05-I-1-B,2005-06-28,None,None,None,27,8
2,WUR:CE-FIN05-I-1-C,2005-06-28,None,None,None,35,4
3,WUR:CE-FIN05-I-2-A,2005-06-28,None,None,None,25,7
4,WUR:CE-FIN05-I-2-B,2005-06-28,None,None,None,None,1
5,WUR:CE-FIN05-I-2-C,2005-06-28,None,None,None,26,8
6,WUR:CE-FIN05-I-3-A,2005-06-28,None,None,None,28,4
7,WUR:CE-FIN05-I-3-B,2005-06-28,None,None,None,None,None
8,WUR:CE-FIN05-I-3-C,2005-06-28,None,None,None,27,6
9,WUR:CE-FIN05-I-4-A,2005-06-28,None,None,None,24,11


In [23]:
# look at data info
df_2005Fin;
df_2005Fin.dtypes;
df_2005Fin.info();
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(df_2004FinNL)

# examining missing values
print('Are there any missing values? ', df_2005Fin.isnull().values.any())
print('Are there any missing values in the Date column? ',df_2005Fin['Date'].isnull().values.any())
print('How many missing values are there per column?\n', df_2005Fin.isnull().sum())
missing = df_2005Fin.isnull().sum().sum()
print('How many missing values are there overall? ', missing)
total_cells = len(df_2005Fin.columns) * len(df_2005Fin)
print('Total cells: ' + str(total_cells))
print('Existing values: ' + str(total_cells - missing))
print('Missing value percentage: ' + str( 
      round(missing * 100 / total_cells,2)) + '%')

# g.remove((None, None, None))
g = parseStudyData('data_2005Fin.csv', df_2005Fin, nodes)
#print(g_2004Fin.serialize(format='ttl').decode('utf8'))
g.serialize(destination=data_gen + 'data_2005Fin.ttl', format='ttl');

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3087 entries, 0 to 3086
Data columns (total 7 columns):
Observation unit ID     3087 non-null object
Date                    3087 non-null object
NumMainStems            766 non-null object
TubN_total_per_plant    1965 non-null object
TubW_total_per_plant    1961 non-null object
DoE                     3003 non-null object
PlantHeight             2254 non-null object
dtypes: object(7)
memory usage: 84.4+ KB
Are there any missing values?  True
Are there any missing values in the Date column?  False
How many missing values are there per column?
 Observation unit ID        0
Date                       0
NumMainStems            2321
TubN_total_per_plant    1122
TubW_total_per_plant    1126
DoE                       84
PlantHeight              833
dtype: int64
How many missing values are there overall?  5486
Total cells: 21609
Existing values: 16123
Missing value percentage: 25.39%
